In [1]:
import xgboost as xgb
import numpy as np
import json
import pandas as pd# Used for Regression Modelling
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pickle

In [2]:
data = pd.read_csv('./data/merged_mldata2020v2.csv',delimiter = ',')
with open(f'score_predictor.pkl', 'rb') as f:
    model = pickle.load(f)

outputs = []
games = data.drop(columns =['vscore', 'hscore','winner','ou',"sWinner","fspread","fscore","vospread","vcspread"])
games = pd.DataFrame(games)

weekfilter = games['week'] == 1

games = games[weekfilter]
#print(games)
columnheads = ['temp','wind_mph','vdflg','hdflg','divgame','nsite','hospread','ouopen','hcspread','ouclose','vTOTAL.DVOA','vTOTAL.RNK','vOFF.RNK','vOFF.DVOA','vDEF.RNK','vDEF.DVOA','vST.RNK','vST.DVOA','hTOTAL.DVOA','hTOTAL.RNK','hOFF.RNK','hOFF.DVOA','hDEF.RNK','hDEF.DVOA','hST.RNK','hST.DVOA','vtsw','vtsl','vtst','htsw','htsl','htst','vtw','vtl','vtt','vts','htw','htl','htt','hts']

games = np.hsplit(games,[3])

print(games)
tw = games[0]
pdata = games[1]



input_variables = pd.DataFrame(pdata, columns = columnheads,dtype=float)

#prediction = model.predict(input_variables)

input_variables["prediction"] = model.predict(input_variables)

#print(prediction)
#print(input_variables)

output_info = input_variables[["hcspread","ouclose","prediction"]]
output_info = tw.join(output_info)

output_info["vcspread"] = np.where(output_info["hcspread"] >= 0,"-" + output_info["hcspread"].astype(str),output_info["hcspread"].astype(str).str.replace("-","+"))
output_info["hcspread"] = np.where(output_info["hcspread"] >= 0,"+" + output_info["hcspread"].astype(str),output_info["hcspread"])


output_info["prediction"] = np.where(output_info["prediction"] < 0, output_info["vteam"] +" "+ output_info["vcspread"].astype(str) , output_info["hteam"] +" "+ output_info["hcspread"].astype(str))
result = output_info.to_json(orient="table")

parsed = json.loads(result)

#json.dumps(parsed, indent=10)  
print(parsed["data"][0])
print(parsed["data"][4])


#for i in games.iterrows():
    #prediction = {'vteam':i['vteam'],'hteam':i['hteam']}
    #input_variables = x.drop(columns=['week','vteam','hteam'])
    #prediction['outcome'] = model.predict(input_variables)[0]
    #print(i['vteam'])


#print(x)
#print(teams)

[   week vteam hteam
0     1   HOU    KC
1     1   MIA    NE
2     1   CLE   BAL
3     1   NYJ   BUF
4     1    LV   CAR
5     1   SEA   ATL
6     1   PHI   WAS
7     1   CHI   DET
8     1   IND   JAX
9     1    GB   MIN
10    1   LAC   CIN
11    1   ARI    SF
12    1    TB    NO
13    1   DAL   LAR
14    1   PIT   NYG
15    1   TEN   DEN,    temp wind_mph vdflg hdflg divgame nsite hospread ouopen hcspread ouclose  \
0    58        6    -1    -1       0     0     -9.5     54     -9.5    53.5   
1    73        6    -1    -1       1     0       -7     43       -7    41.5   
2    75        5    -1    -1       1     0     -8.5   48.5       -7    47.5   
3    65       12    -1    -1       1     0       -6   39.5     -6.5      39   
4    79        3    -1    -1       0     0      1.5     47        3      48   
5    72        0    -1    -1       0     0       -1   49.5        1    49.5   
6    76        4    -1    -1       1     0      6.5     44      5.5    41.5   
7    72        0    -1    